<a href="https://colab.research.google.com/github/MohsenJadidi/Erdos-Goes-Neural/blob/main/ModifiedCode/Clique_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [ ]:
#!pip install torch_scatter==latest+cu101 torch_sparse==latest+cu101
#!pip install torch_geometric


In [1]:
!pip install torch_scatter
!pip install torch_sparse
!pip install torch_geometric


In [2]:
!pip install graphviz
!pip install pygraphviz

In [3]:
!pip install visdom
!pip install GPUtil

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('/content/drive/My Drive/ISLR Course - Fall 2020/Project')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Import

In [9]:
import os
import pickle
import torch
import torch.nn.functional as F
from torch.nn import Linear
#from kernel.datasets import get_dataset
import time
from torch import tensor
from torch.optim import Adam
from torch.optim import SGD
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from math import ceil
from torch.nn import Linear
from torch.distributions import categorical
from torch.distributions import Bernoulli
import torch.nn
%matplotlib inline
from matplotlib import pyplot as plt
import pygraphviz as pgv
from torch_geometric.utils import convert as cnv
from torch_geometric.utils import sparse as sp
from torch_geometric.data import Data
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
from torch.utils.data.sampler import RandomSampler
from torch.nn.functional import gumbel_softmax
from torch.distributions import relaxed_categorical
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.inits import glorot, zeros
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
from torch_geometric.nn import GINConv, GATConv
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
#from torch_geometric.utils import scatter_
from torch_geometric.data import Batch 
from torch_scatter import scatter_min, scatter_max, scatter_add, scatter_mean
from torch import autograd
from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops
from models import cut_MPNN
from modules_and_utils import derandomize_cut, GATAConv,get_diracs, derandomize_clique_final, derandomize_clique_final_speed, solve_gurobi_maxclique
import scipy
import scipy.io
import GPUtil
from networkx.algorithms.approximation import max_clique
from torch_geometric.data import DataListLoader
from random import shuffle
from networkx.algorithms.approximation import max_clique
from networkx.algorithms import graph_clique_number
from networkx.algorithms import find_cliques

# Read Dataset

In [10]:
datasets = ["TWITTER_SNAP", "COLLAB", "IMDB-BINARY"]
dataset_name = datasets[2]
path_to_dataset ="%PATH"
if dataset_name == "TWITTER_SNAP":
    stored_dataset = open(path_to_dataset, 'rb')        
elif dataset_name == "COLLAB":
    stored_dataset = open(path_to_dataset, 'rb')
elif dataset_name == "IMDB-BINARY":
    stored_dataset = open(path_to_dataset, 'rb')

address = '/content/drive/My Drive/ISLR Course - Fall 2020/Project/IMDB_BINARY.p'
stored_dataset = open(address, 'rb')

dataset = pickle.load(stored_dataset)
dataset_scale = 1
total_samples = int(np.floor(len(dataset)*dataset_scale))
dataset = dataset[:total_samples]

num_trainpoints = int(np.floor(0.6*len(dataset)))
num_valpoints = int(np.floor(num_trainpoints/3))
num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)
traindata= dataset[0:num_trainpoints]
valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]
testdata = dataset[num_trainpoints + num_valpoints:]
batch_size = 32
train_loader = DataLoader(traindata, batch_size, shuffle=True)
test_loader = DataLoader(testdata, batch_size, shuffle=False)
val_loader =  DataLoader(valdata, batch_size, shuffle=False)

#set up random seeds 
torch.manual_seed(1)
np.random.seed(2)   
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

FileNotFoundError: ignored